# Scraping Toronto's Postal Codes Table from Wikipedia

### Import libraries to Scrape the website

In [1]:
from bs4 import BeautifulSoup
import requests
print("Libraries imported!")
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Libraries imported!


### Get the table html

In [2]:
with requests.Session() as s:
    r = s.get(url)
    #print(r.text)
    soup = BeautifulSoup(r.text, 'html.parser')
    #print(soup.prettify())
    table = soup.find('table',{'class':'wikitable sortable'})
    

### Method  to remove "\n" from the string

In [4]:
import re

def remove_n_char(s):
    return re.sub('\n', '', s)

### Get the table columns

In [5]:
header = []
theader = table.find_all('th')
for row in range(len(theader)):
    #print(theader[row].string)
    header.append(theader[row].string)
print(header)

['Postal Code\n', 'Borough\n', 'Neighborhood\n']


In [6]:
for col in range(len(header)):
    header[col] = remove_n_char(header[col])
header

['Postal Code', 'Borough', 'Neighborhood']

### Get the table rows

In [7]:
tr = table.find_all('tr')
data = []
for row in tr:
    data.append(row.find_all('td'))

In [8]:
for row in range(len(data)):
    for column in range(len(data[row])):
        if len(data[row]) > 0:
            data[row][column] = remove_n_char(data[row][column].string)
#data

### Transform the list in a Pandas Dataframe

In [9]:
import pandas as pd
df = pd.DataFrame(data)

In [10]:
df.tail()

,0,1,2
176,M5Z,Not assigned,Not assigned
177,M6Z,Not assigned,Not assigned
178,M7Z,Not assigned,Not assigned
179,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
180,M9Z,Not assigned,Not assigned


In [11]:
df.columns = header

In [12]:
df.tail()

,Postal Code,Borough,Neighborhood
176,M5Z,Not assigned,Not assigned
177,M6Z,Not assigned,Not assigned
178,M7Z,Not assigned,Not assigned
179,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
180,M9Z,Not assigned,Not assigned


## Dataframe processing instructions

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [13]:
df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [14]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [15]:
import numpy as np
df.fillna(value=np.nan, inplace=True)
df.dropna(inplace=True)

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [16]:
#df[(df["Borough"] != 'None') & (df["Borough"] != "Not assigned")]

df = df[df["Borough"] != "Not assigned"]

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in the above table.

In [17]:
df[df["PostalCode"] == "M5A"]

,PostalCode,Borough,Neighborhood
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [18]:
df[df["Neighborhood"] == "Not assigned"]

,PostalCode,Borough,Neighborhood


In [19]:
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [20]:
df.shape

(103, 3)

In [21]:
#conda install -c conda-forge geocoder

### Code to get the latitudes and longitudes via geocoder:

import geocoder # import geocoder

latitudes = []
longitudes = []

for postal_code in df["PostalCode"]:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    latitudes.append(latitude)
    longitudes.append(longitude)

#### Since the package is not reliable, I am using the url https://cocl.us/Geospatial_data to get the data

In [22]:
postalCode_df = pd.read_csv("https://cocl.us/Geospatial_data")

In [23]:
postalCode_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [24]:
postalCode_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
result = pd.merge(df, postalCode_df, on='PostalCode')

In [26]:
result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
